In [2]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import requests as r
from datetime import datetime, timedelta
from typing import Tuple, List, Dict
from time import sleep
import os
from tqdm.notebook import tqdm

vk_vers = 5.131 # current vk version
vk_access_token='YOUR_API_KEY'

# Функции 

## Сбор параметров для запроса

In [2]:
def get_data(coords: Tuple[float, float], dist:int, timeperiod: Tuple[int, int], offset: int = 0) -> Dict:
    """
    Выполнить GET-запрос к API ВКонтакте для поиска фотографий.

    Args:
        coords (Tuple[float, float]): Координаты поиска (широта, долгота).
        coords (int): Радиус поиска в м.
        timeperiod (Tuple[int, int]): Временной промежуток в формате Unix timestamp (начало, конец).
        offset (int): Смещение для пагинации данных.

    Returns:
        dict: JSON-ответ от VK API.
    """
    params = {
        'lat': coords[0],
        'long': coords[1],
        'count': 1000,
        'offset': offset,
        'radius': dist,
        'start_time': timeperiod[0],
        'end_time': timeperiod[1],
        'access_token': vk_access_token,
        'v': vk_vers,
        'sort': 0  # сортировка по дате создания
    }
    try:
        response = r.get("https://api.vk.com/method/photos.search", params=params, verify=True)
        response.raise_for_status()
        return response.json()
    except r.RequestException as e:
        print(f"Ошибка запроса данных: {e}")
        return {}

## Сохранение ответа от сервера VK

In [3]:
def save_points(response: Dict, df_points: pd.DataFrame) -> None:
    """
    Сохранить данные из ответа VK API в DataFrame.

    Args:
        response (dict): Ответ от VK API.
        df_points (pd.DataFrame): DataFrame для сохранения точек.
    """
    try:
        items = response['response']['items']
    except KeyError:
        print("Ключ 'items' отсутствует в ответе.")
        return
    
    for item in items:
        try:
            record = {
                'id': item['id'],
                'owner_id': item['owner_id'],
                'url': item['sizes'][-1]['url'],
                'date': datetime.fromtimestamp(item['date']),
                'text': item.get('text', ''),
                'lat': item.get('lat', None),
                'long': item.get('long', None)
            }
            df_points.loc[len(df_points)] = record
        except KeyError as e:
            print(f"Ошибка при обработке элемента: {e}")
            continue

## Сбор данных в GeoDataFrame

In [4]:
def collect_data(coords: Tuple[float, float], dist: int, start_date: datetime, end_date: datetime, delete_groups = False) -> gpd.GeoDataFrame:
    """
    Собрать данные из VK API в заданном временном промежутке.

    Args:
        coords (Tuple[float, float]): Координаты поиска (широта, долгота).
        dist (int): Радиус поиска в метрах.
        start_date (datetime): Дата начала поиска.
        end_date (datetime): Дата конца поиска.
        delete_groups (bool): Удалить группы - да/нет.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame с собранными данными.
    """
    # Определяем шаг в 1 день (в секундах)
    step = timedelta(days=1).total_seconds()

    # Создаем список временных интервалов (начало каждого дня)
    time_intervals = list(range(int(start_date.timestamp()), int(end_date.timestamp()), int(step)))

    # Инициализируем DataFrame для хранения результатов
    df_points = pd.DataFrame(columns=['id', 'owner_id', 'url', 'date', 'text', 'lat', 'long'])

    # Используем tqdm для отображения прогресса
    for current_time in tqdm(time_intervals, desc="Обработка временных интервалов"):
        # Запрос данных за 1 день
        resp = get_data(coords, dist, (current_time, current_time + step), offset=0)
        save_points(resp, df_points)

        # Если данных больше, чем 1000, обрабатываем их с `offset`
        try:
            count = resp['response']['count']
            returned = len(resp['response']['items'])
        except KeyError:
            print(f"Пропущен временной интервал: {datetime.fromtimestamp(current_time)}")
            sleep(0.5)  # Задержка перед переходом к следующему интервалу
            continue

        offset = returned
        while offset < count and offset < 3000:  # Ограничение API
            resp = get_data(coords, dist, (current_time, current_time + step), offset)
            save_points(resp, df_points)
            offset += len(resp['response']['items'])

        # Добавляем задержку между запросами
        sleep(0.5)

    # Преобразуем DataFrame в GeoDataFrame
    gdf_points = gpd.GeoDataFrame(
        df_points,
        geometry=gpd.points_from_xy(df_points.long, df_points.lat),
        crs="EPSG:4326"
    )
    gdf_points = gdf_points[~gdf_points.geometry.is_empty]

    if delete_groups:
        gdf_points = gdf_points.loc[~gdf_points['owner_id'].astype(str).str.startswith('-')]

    return gdf_points

# Использование скрипта

In [ ]:
coords = (53.361637, 55.924653)  # Широта, Долгота
dist = 5000  # Радиус поиска в м.
start_date = datetime(2022, 1, 1)  # Стартовая дата поиска
end_date = datetime(2024, 12, 1) # Конечная дата поиска 
delete_groups = False # Удалить группы? Нет/Да

gdf_points = collect_data(coords, dist, start_date, end_date)

# Визуализация точек на тепловой карте

In [ ]:
osm_map = folium.Map(location=[gdf_points.lat.mean(), gdf_points.long.mean()], tiles="cartodb positron", zoom_start=14)
map_matrix = gdf_points[['lat', 'long']].values
HeatMap(map_matrix).add_to(osm_map)
osm_map